<a href="https://colab.research.google.com/github/sarveshdighe38/Audio-to-Text-Translation-System/blob/main/LASTCODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import soundfile as sf
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from glob import glob
import time
from jiwer import wer
import torchaudio.transforms as T
from sklearn.model_selection import train_test_split
import pickle
import json

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
num_classes = 29  # a-z + space + blank+'
lr = 0.001 #learning rate (how large the update step is)
epochs =256 #how many times we want to interate
sample_rate = 16000
n_mels = 80
batch_size = 8
data_root = "/content/drive/MyDrive/ASR/new_dev_clean"
#checkpoint_path = '/content/drive/MyDrive/Augmented_BiLSTM_ASR_1988.pth'
test_audio ="/content/drive/MyDrive/ASR/TESTING_SET/3170/137823/3170-137823-0013.flac"
test_audio_text="WE'D BETTER NOT TRY TO MOVE HIM TOM DECIDED WE'LL GET AN AMBULANCE"
test_audio_text=test_audio_text.lower()

In [ ]:
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=512,
    hop_length=160,
    win_length=400,
    n_mels=n_mels
)


In [ ]:
char_map_str = [" "] + list("abcdefghijklmnopqrstuvwxyz") + ["'", "<blank>"]
char_to_idx = {c:i for i,c in enumerate(char_map_str)}
blank_idx = num_classes - 1

In [ ]:
flac_files = sorted(glob(os.path.join(data_root, "**", "*.flac"), recursive=True))
print("Found FLAC files:", len(flac_files))
print(flac_files)

Found FLAC files: 196
['/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0000.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0001.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0002.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0003.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0004.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0005.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0006.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0007.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0008.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0009.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0010.flac', '/content/drive/MyDrive/ASR/new_dev_clean/1462/170138/1462-170138-0011.flac', '/content/drive/MyDrive/ASR/new_dev_clean

In [ ]:
transcripts = {}
for trans_path in glob(os.path.join(data_root, "**", "*.trans.txt"), recursive=True):
    with open(trans_path, "r") as f:
        for line in f:
            fname, *text = line.strip().split()
            transcripts[fname] = " ".join(text)

print("Loaded transcripts:", len(transcripts))
print(transcripts)

Loaded transcripts: 196
{'1462-170145-0000': "ON THE LAST SATURDAY IN APRIL THE NEW YORK TIMES PUBLISHED AN ACCOUNT OF THE STRIKE COMPLICATIONS WHICH WERE DELAYING ALEXANDER'S NEW JERSEY BRIDGE AND STATED THAT THE ENGINEER HIMSELF WAS IN TOWN AND AT HIS OFFICE ON WEST TENTH STREET", '1462-170145-0001': 'OVER THE FIREPLACE THERE WAS A LARGE OLD FASHIONED GILT MIRROR', '1462-170145-0002': 'HE ROSE AND CROSSED THE ROOM QUICKLY', '1462-170145-0003': "OF COURSE I KNOW BARTLEY SHE SAID AT LAST THAT AFTER THIS YOU WON'T OWE ME THE LEAST CONSIDERATION BUT WE SAIL ON TUESDAY", '1462-170145-0004': "I SAW THAT INTERVIEW IN THE PAPER YESTERDAY TELLING WHERE YOU WERE AND I THOUGHT I HAD TO SEE YOU THAT'S ALL GOOD NIGHT I'M GOING NOW", '1462-170145-0005': "LET ME TAKE OFF YOUR COAT AND YOUR BOOTS THEY'RE OOZING WATER", '1462-170145-0006': "IF YOU'D SENT ME A NOTE OR TELEPHONED ME OR ANYTHING", '1462-170145-0007': 'I TOLD MYSELF THAT IF I WERE REALLY THINKING OF YOU AND NOT OF MYSELF A LETTER WOULD B

In [ ]:
missing = []
for f in flac_files:
    name = f.split('/')[-1].replace('.flac', '')
    if name not in transcripts:
        missing.append(name)

if missing:
    print("Missing transcripts for:", missing)
else:
    print("All audio files have transcripts")

All audio files have transcripts


In [ ]:
def text_to_labels(text):
    return [char_to_idx[c] for c in text.lower() if c in char_to_idx]

def load_audio(path, target_sr=16000, training=True):
    waveform, sr = sf.read(path, dtype="float32")
    waveform = torch.from_numpy(waveform)

    if waveform.ndim > 1:
        waveform = waveform.mean(dim=1)

    waveform = waveform.unsqueeze(0)  # [1, T]

    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)

    max_val = waveform.abs().max()
    if max_val > 0:
        waveform = waveform / max_val

    # Speed perturbation (TRAINING ONLY)
    if training and torch.rand(1) < 0.5:
        rate = 0.9 if torch.rand(1) < 0.5 else 1.1
        new_sr = int(target_sr * rate)

        waveform = torchaudio.functional.resample(
            waveform, target_sr, new_sr
        )

        waveform = torchaudio.functional.resample(
            waveform, new_sr, target_sr
        )

    return waveform, target_sr



In [ ]:
class TemporalCNN_BiLSTM(nn.Module):
    def __init__(self, num_classes, lstm_hidden=256, lstm_layers=2):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.pool1 = nn.MaxPool2d((2,2), stride=(2,1))
        self.pool2 = nn.MaxPool2d((2,2), stride=(2,2))


        self.lstm = nn.LSTM(
            input_size=64 * 20,   # 64 *freq/4
            hidden_size=256,
            num_layers=2,
            bidirectional=True,
            batch_first=True
        )

        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(256 * 2, num_classes)

    def forward(self, x):

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        B, C, Freq, T = x.shape

        x = x.permute(0, 3, 1, 2).contiguous()
        x = x.view(B, T, C * Freq)

        x, _ = self.lstm(x)
        #x = self.dropout(x)

        x = self.fc(x)

        return x


In [ ]:
model = TemporalCNN_BiLSTM(num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3)
ctc_loss = nn.CTCLoss(blank=blank_idx, zero_infinity=True)

In [ ]:

class ASRDataset(Dataset):
    def __init__(self, flac_files, transcripts, mel_transform, training=True):
        self.flac_files = flac_files
        self.transcripts = transcripts
        self.mel_transform = mel_transform
        self.training = training

    def __len__(self):
        return len(self.flac_files)

    def __getitem__(self, idx):
        flac_path = self.flac_files[idx]
        name = flac_path.split('/')[-1].replace('.flac', '')

        waveform, _ = load_audio(flac_path, training=self.training)

        mel_spec = self.mel_transform(waveform)  # [1, n_mels, T]

        # SpecAugment (TRAINING ONLY)
        if self.training:
            n_mels = mel_spec.shape[1]
            num_frames = mel_spec.shape[2]

            freq_mask = int(n_mels * 0.2)
            time_mask = max(1, int(num_frames * 0.05))

            mel_spec = T.FrequencyMasking(freq_mask)(mel_spec)
            mel_spec = T.TimeMasking(time_mask)(mel_spec)

        mel_spec = (mel_spec - mel_spec.mean()) / (mel_spec.std() + 1e-5)

        label = torch.tensor(
            text_to_labels(self.transcripts[name]),
            dtype=torch.long
        )

        return mel_spec.squeeze(0), label  # [80, T], [L]



In [ ]:
def collate_fn(batch):
        batch = sorted(batch, key=lambda x: x[0].shape[1], reverse=True)
        specs, labels = zip(*batch)

        max_len = specs[0].shape[1]
        padded_specs = []
        input_lengths = []
        for spec in specs:
          T = spec.shape[1]
          pad = max_len - T
          padded = F.pad(spec, (0, pad))
          padded_specs.append(padded)
          input_lengths.append(T)
        padded_specs = torch.stack(padded_specs)  # [B, 80, max_T]

        # CNN time reduction = 2× with floor rounding
        input_lengths = torch.tensor([(T // 2) for T in input_lengths], dtype=torch.long)



        targets = torch.cat(labels)
        target_lengths = torch.tensor([len(l) for l in labels], dtype=torch.long)

        return padded_specs.unsqueeze(1), targets, input_lengths, target_lengths

In [ ]:
dataset = ASRDataset(flac_files, transcripts, mel_transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
print(len(loader))

25


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=42)

In [ ]:

'''fold_results = []
for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n===== Fold {fold+1} =====")

    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset   = torch.utils.data.Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader   = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


    model = TemporalCNN_BiLSTM(num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3)
    ctc_loss = nn.CTCLoss(blank=blank_idx, zero_infinity=True)

    checkpoint_path = f"/content/drive/MyDrive/WEIGHTS/checkpoint_fold_{fold}.pt"
    best_val_loss = float("inf")

    train_losses = []
    start = time.time()

  for epoch in range(start_epoch, epochs):
    total_loss = 0.0
    model.train()
    for mel_specs, targets, input_lengths, target_lengths in loader:
        mel_specs = mel_specs.to(device)
        targets = targets.to(device)
        input_lengths = input_lengths.to(device)
        target_lengths = target_lengths.to(device)

        optimizer.zero_grad()# removes grdient of previous

        logits = model(mel_specs)            # [B, T, C]
        log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2) # [T, B, C] permute because ctc_loss required input in this format and dim=-1 because we apply softmax on num_class

        input_lengths = torch.clamp(input_lengths, max=log_probs.size(0))

        loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"NaN/Inf loss in batch, skipping")
            continue

        loss.backward()#calculate change in loss means gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()#changes lr accordingly

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    train_losses.append(avg_loss)
    if epoch % 10 ==0
      model.eval()
      val_loss = 0.0

      with torch.no_grad():
        for mel_specs, targets, input_lengths, target_lengths in val_loader:
          mel_specs = mel_specs.to(device)
          targets = targets.to(device)
          input_lengths = input_lengths.to(device)
          target_lengths = target_lengths.to(device)

          logits = model(mel_specs)
          log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2)
          input_lengths = torch.clamp(input_lengths, max=log_probs.size(0))

          loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
          val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Fold {fold} | Epoch {epoch} | Train {avg_train_loss:.4f} | Val {avg_val_loss:.4f}")

      if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict()
        }, checkpoint_path)



  end = time.time()
  print(f"Training finished in {(end - start):.2f}s")'''

No checkpoint found. Starting from scratch.
Epoch 0/501 - Loss: 2.9434510
Saved best model
Epoch 1/501 - Loss: 2.9471322
Epoch 2/501 - Loss: 2.9405198
Epoch 3/501 - Loss: 2.9396995
Epoch 4/501 - Loss: 2.9461818
Epoch 5/501 - Loss: 2.9280551
Epoch 6/501 - Loss: 2.9239875
Epoch 7/501 - Loss: 2.9174115
Epoch 8/501 - Loss: 2.9096893
Epoch 9/501 - Loss: 2.9007622
Epoch 10/501 - Loss: 2.8939179
Epoch 11/501 - Loss: 2.8924230
Epoch 12/501 - Loss: 2.8931498
Epoch 13/501 - Loss: 2.8957200
Epoch 14/501 - Loss: 2.8707672
Epoch 15/501 - Loss: 2.8777984
Epoch 16/501 - Loss: 2.8674435
Epoch 17/501 - Loss: 2.8450582
Epoch 18/501 - Loss: 2.8319470
Epoch 19/501 - Loss: 2.8170928
Epoch 20/501 - Loss: 2.7827404
Epoch 21/501 - Loss: 2.7539379
Epoch 22/501 - Loss: 2.7302568
Epoch 23/501 - Loss: 2.6744871
Epoch 24/501 - Loss: 2.6463728
Epoch 25/501 - Loss: 2.5895998
Epoch 26/501 - Loss: 2.5443362
Epoch 27/501 - Loss: 2.5012721
Epoch 28/501 - Loss: 2.4360530
Epoch 29/501 - Loss: 2.3955491
Epoch 30/501 - Loss

In [ ]:
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n===== Fold {fold+1} =====")

    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset   = torch.utils.data.Subset(dataset, val_idx)

    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    model = TemporalCNN_BiLSTM(num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3
    )
    ctc_loss = nn.CTCLoss(blank=blank_idx, zero_infinity=True)

    checkpoint_path = f"/content/drive/MyDrive/new_checkpoints_fold_{fold}.pt"

    if os.path.exists(checkpoint_path):
      print(f"Fold {fold+1} already trained, skipping")
      continue
    best_val_loss = float("inf")

    start_epoch = 0
    start_time = time.time()
    train_losses=[]
    val_losses=[]
    for epoch in range(start_epoch, epochs):

        # -------- TRAIN --------
        model.train()
        train_loss = 0.0

        for mel_specs, targets, input_lengths, target_lengths in train_loader:
            mel_specs = mel_specs.to(device)
            targets = targets.to(device)
            input_lengths = input_lengths.to(device)
            target_lengths = target_lengths.to(device)

            optimizer.zero_grad()

            logits = model(mel_specs)
            log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2)
            input_lengths = torch.clamp(input_lengths, max=log_probs.size(0))

            loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"NaN/Inf loss in batch, skipping")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # -------- VALIDATE --------
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for mel_specs, targets, input_lengths, target_lengths in val_loader:
                mel_specs = mel_specs.to(device)
                targets = targets.to(device)
                input_lengths = input_lengths.to(device)
                target_lengths = target_lengths.to(device)

                logits = model(mel_specs)
                log_probs = F.log_softmax(logits, dim=-1).permute(1, 0, 2)
                input_lengths = torch.clamp(input_lengths, max=log_probs.size(0))

                loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        scheduler.step(avg_val_loss)

        print(
            f"Fold {fold+1} | Epoch {epoch+1} | "
            f"Train {avg_train_loss:.4f} | Val {avg_val_loss:.4f}"
        )

        # -------- SAVE BEST --------
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'fold': fold,
                'epoch': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict()
            }, checkpoint_path)

    elapsed = time.time() - start_time
    print(f"Fold {fold+1} finished in {elapsed:.2f}s")

    fold_results.append(best_val_loss)

    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', label="Train Loss")

    'if len(val_losses) > 0:
      plt.plot(range(1, len(val_losses) + 1), val_losses, marker='s', label="Val Loss")

    plt.title(f"Training Loss vs Epochs (Fold {fold+1})")
    plt.xlabel("Epoch")
    plt.ylabel("CTC Loss")
    plt.grid(True)
    plt.legend()

    plot_path = f"loss_plots/fold_{fold+1}_loss.png"
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"Saved plot → {plot_path}")



===== Fold 1 =====
Fold 1 already trained, skipping

===== Fold 2 =====
Fold 2 already trained, skipping

===== Fold 3 =====
Fold 3 | Epoch 1 | Train 3.7595 | Val 2.9810
Fold 3 | Epoch 2 | Train 3.0418 | Val 3.0061
Fold 3 | Epoch 3 | Train 2.9966 | Val 2.9719
Fold 3 | Epoch 4 | Train 3.0064 | Val 2.9678
Fold 3 | Epoch 5 | Train 3.0025 | Val 2.9479
Fold 3 | Epoch 6 | Train 2.9755 | Val 2.9458
Fold 3 | Epoch 7 | Train 2.9701 | Val 2.9468
Fold 3 | Epoch 8 | Train 2.9597 | Val 2.9389
Fold 3 | Epoch 9 | Train 2.9513 | Val 2.9293
Fold 3 | Epoch 10 | Train 2.9602 | Val 2.9463
Fold 3 | Epoch 11 | Train 2.9416 | Val 2.9480
Fold 3 | Epoch 12 | Train 2.9192 | Val 2.9065
Fold 3 | Epoch 13 | Train 2.9044 | Val 2.8951
Fold 3 | Epoch 14 | Train 2.8876 | Val 2.8868
Fold 3 | Epoch 15 | Train 2.8799 | Val 2.8847
Fold 3 | Epoch 16 | Train 2.8730 | Val 2.8797
Fold 3 | Epoch 17 | Train 2.8554 | Val 2.8765
Fold 3 | Epoch 18 | Train 2.8237 | Val 2.8355
Fold 3 | Epoch 19 | Train 2.7881 | Val 2.8074
Fold 3 | 

In [ ]:
  def ctc_greedy_decode(pred_ids, char_map_str, blank_idx):
    pred_ids = pred_ids.tolist()
    decoded = []
    prev = blank_idx
    for p in pred_ids:
        if p != prev and p != blank_idx:
            decoded.append(char_map_str[p])
        prev = p
    return "".join(decoded)


In [ ]:
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state'])

model.eval()

waveform,sr = load_audio(test_audio,target_sr=16000,training=False)
print("Waveform shape:", waveform.shape)  # must be [1, T]

sample_mel = mel_transform(waveform)
print("Mel shape:", sample_mel.shape)     # [1, n_mels, time]

sample_mel = (sample_mel - sample_mel.mean()) / (sample_mel.std() + 1e-5)

sample_mel = sample_mel.unsqueeze(0).to(device)  # [1, 1, Freq, Time]

with torch.no_grad():
    logits = model(sample_mel)
    predicted_ids = torch.argmax(logits, dim=-1).cpu()

decoded_text = ctc_greedy_decode(predicted_ids[0], char_map_str, blank_idx)
print("Predicted text:", decoded_text)


Waveform shape: torch.Size([1, 79360])
Mel shape: torch.Size([1, 80, 497])
Predicted text: we'd better not try to move him tom decided we'll get an ambulance


In [ ]:
wer_score = wer(test_audio_text, decoded_text)
print("WER Score:", wer_score)

WER Score: 0.0


In [ ]:

plt.figure(figsize=(8,5))
plt.plot(range(1, epochs + 1), train_losses, marker='o')
plt.title("Training Loss vs Epochs (Fixed LR)")
plt.xlabel("Epoch")
plt.ylabel("CTC Loss")
plt.grid(True)
plt.show()

NameError: name 'train_losses' is not defined

<Figure size 800x500 with 0 Axes>

In [ ]:
!pip install nemo-toolkit[asr]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of numba to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.5/866.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
!pip uninstall -y numpy
!pip install numpy==2.1.3


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 69.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


In [ ]:
from nemo.collections.asr.models import ASRModel

model = ASRModel.from_pretrained(model_name="stt_en_conformer_ctc_small")
print(model)


[NeMo W 2026-01-13 05:03:59 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.
[NeMo W 2026-01-13 05:04:00 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2026-01-13 05:04:00 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2026-01-13 05:04:00 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2026-01-13 05:04:00 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


[NeMo I 2026-01-13 05:04:01 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small/versions/1.6.0/files/stt_en_conformer_ctc_small.nemo to /root/.cache/torch/NeMo/NeMo_2.6.1/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2026-01-13 05:04:03 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2026-01-13 05:04:04 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2026-01-13 05:04:05 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2026-01-13 05:04:05 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librisp

[NeMo I 2026-01-13 05:04:05 nemo_logging:393] PADDING: 0
[NeMo I 2026-01-13 05:04:05 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.6.1/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.
EncDecCTCModelBPE(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConformerEncoder(
    (pre_encode): ConvSubsampling(
      (out): Linear(in_features=3520, out_features=176, bias=True)
      (conv): MaskedConvSequential(
        (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
    )
    (pos_enc): RelPositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-15): 16 x ConformerLayer(
        (norm_feed_forward1): LayerNorm(

In [ ]:
pretrainM_root = "/content/drive/MyDrive/ASR/dev_clean"
save_model = "/content/drive/MyDrive/ASR/asr_conformer_finetuned.nemo"

In [ ]:
train_files, val_files = train_test_split(
    pretrainM_flac_files,
    test_size=0.15,
    random_state=42
)
len(train_files), len(val_files)

(1125, 199)

In [ ]:
pretrainM_flac_files = sorted(glob(os.path.join(pretrainM_root, "**", "*.flac"), recursive=True))
print("Found FLAC files:", len(pretrainM_flac_files))
print(pretrainM_flac_files)

Found FLAC files: 1324
['/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0000.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0001.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0002.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0003.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0004.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0005.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0006.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0007.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0008.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0009.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0010.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0011.flac', '/content/drive/MyDrive/ASR/dev_clean/1272/135031/1272-135031-0012.flac', '/content/driv

In [ ]:
Pre_transcripts = {}
for trans_path in glob(os.path.join(pretrainM_root, "**", "*.trans.txt"), recursive=True):
    with open(trans_path, "r") as f:
        for line in f:
            fname, *text = line.strip().split()
            Pre_transcripts[fname] = " ".join(text)

print("Loaded transcripts:", len(Pre_transcripts))
print(Pre_transcripts)

Loaded transcripts: 1324
{'6345-93302-0000': 'SHE HERSELF SHOULD HAVE BEEN A POEM A LYRIC IN A WHITE GOWN AND GREEN SCARF COMING TO HIM THROUGH THE LONG GRASS UNDER THE BLOSSOMED BOUGHS', '6345-93302-0001': 'HER HANDS SHOULD HAVE BEEN FULL OF BLUEBELLS AND SHE SHOULD HAVE HELD THEM UP TO HIS FACE IN MAIDENLY DEFENCE AS HE SPRANG FORWARD TO TAKE HER IN HIS ARMS', '6345-93302-0002': 'YOU SEE THAT SHE KNEW EXACTLY HOW A TRYST IS CONDUCTED IN THE PAGES OF THE STANDARD POETS AND OF THE CHEAPER WEEKLY JOURNALS', '6345-93302-0003': 'SHE HAD TO THE FULL LIMIT ALLOWED OF HER READING AND HER ENVIRONMENT THE LITERARY SENSE', '6345-93302-0004': 'AND CURIOUSLY ENOUGH SHE WAS HARDLY CURIOUS AT ALL ABOUT WHAT HE MIGHT HAVE TO SAY', '6345-93302-0005': 'SHE ONLY WISHED FOR MAY AND THE ORCHARD INSTEAD OF JANUARY AND THE DINGY DUSTY WAITING ROOM THE PLAIN FACED PREOCCUPIED TRAVELLERS THE DIM DESOLATE WEATHER', '6345-93302-0006': 'THE SETTING OF THE SCENE SEEMED TO HER ALL IMPORTANT', '6345-93302-0007': '

In [ ]:
missing = []
for f in pretrainM_flac_files:
    name = f.split('/')[-1].replace('.flac', '')
    if name not in Pre_transcripts:
        missing.append(name)

if missing:
    print("Missing transcripts for:", missing)
else:
    print("All audio files have transcripts")

All audio files have transcripts


In [ ]:

with open("training.json", "w") as fout:
    for fname in (train_files):

        utt_id = os.path.basename(fname).replace(".flac", "")

        if utt_id not in Pre_transcripts:
            print("Skipping", utt_id)
            continue

        #duration = librosa.get_duration(path=audio_path)
        info = sf.info(fname)
        duration = info.frames / info.samplerate
        entry = {
            "audio_filepath": fname,
            "duration": round(duration, 3),
            "text": Pre_transcripts[utt_id]
        }

        fout.write(json.dumps(entry) + "\n")

In [ ]:

with open("val.json", "w") as fout:
    for fname in (val_files):

        utt_id = os.path.basename(fname).replace(".flac", "")

        if utt_id not in Pre_transcripts:
            print("Skipping", utt_id)
            continue

        #duration = librosa.get_duration(path=audio_path)
        info = sf.info(fname)
        duration = info.frames / info.samplerate
        entry = {
            "audio_filepath": fname,
            "duration": round(duration, 3),
            "text": Pre_transcripts[utt_id]
        }

        fout.write(json.dumps(entry) + "\n")

In [ ]:
# Training
model.setup_training_data(
    train_data_config={
        "manifest_filepath": "training.json",
        "sample_rate": 16000,
        "batch_size": 8,
        "shuffle": True,
        "num_workers": 2
    }
)

# Validation
model.setup_validation_data(
    val_data_config={
        "manifest_filepath": "val.json",
        "sample_rate": 16000,
        "batch_size": 8,
        "shuffle": False,
        "num_workers": 2
    }
)


[NeMo I 2026-01-12 17:21:10 nemo_logging:393] Dataset loaded with 1125 files totalling 2.24 hours
[NeMo I 2026-01-12 17:21:10 nemo_logging:393] 0 files were filtered totalling 0.00 hours
[NeMo I 2026-01-12 17:21:10 nemo_logging:393] Dataset loaded with 199 files totalling 0.41 hours
[NeMo I 2026-01-12 17:21:10 nemo_logging:393] 0 files were filtered totalling 0.00 hours


In [ ]:
model.setup_optimization(
    optim_config={
        "lr": 1e-4,
        "weight_decay": 1e-5,
        "betas": [0.9, 0.98],
    }
)


[NeMo W 2026-01-12 17:23:35 nemo_logging:405] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2026-01-12 17:23:35 nemo_logging:393] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        decoupled_weight_decay: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0001
        maximize: False
        weight_decay: 1e-05
    )
[NeMo I 2026-01-12 17:23:35 nemo_logging:393] Scheduler not initialized as no `sched` config supplied to setup_optimizer()


(Adam (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     capturable: False
     decoupled_weight_decay: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.0001
     maximize: False
     weight_decay: 1e-05
 ),
 None)

In [ ]:

from lightning.pytorch import Trainer

trainer=Trainer(
    max_epochs=20,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=1.0,
    precision=16,
    log_every_n_steps=10
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
model.set_trainer(trainer)

trainer.fit(model)
model.save_to(save_model)



INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2026-01-12 17:23:52 nemo_logging:393] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        decoupled_weight_decay: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0001
        maximize: False
        weight_decay: 1e-05
    )
[NeMo I 2026-01-12 17:23:52 nemo_logging:393] Scheduler not initialized as no `sched` config supplied to setup_optimizer()


INFO: 
  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoder                  | 13.0 M | train
2 | decoder           | ConvASRDecoder                    | 181 K  | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
13.2 M    Trainable params
0         Non-trainable params
13.2 M    Total params
52.616    Total estimated model params size (MB)
468       Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name              | Type                              | Param

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2026-01-12 17:23:55 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:23:55 nemo_logging:393] WER reference:the court was spacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:23:55 nemo_logging:393] WER predicted:the court was spacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:23:56 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:23:56 nemo_logging:393] WER reference:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them
[NeMo I 2026-01-12 17:23:56 nemo_logging:393] WER predicted:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them


...

Validation: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2026-01-12 17:40:44 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] WER reference:the court was spacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] WER predicted:the court was faceacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] WER reference:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] WER predicted:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them
[NeMo I 2026-01-12 17:40:44 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:40:44 nemo_loggi

Validation: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2026-01-12 17:41:33 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] WER reference:the court was spacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] WER predicted:the court was faceacious well paved and perfectly clean there being probably another entrance behind the stables for removing the litter
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] WER reference:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] WER predicted:miss woodley did not recollect herself so but was so in reality in her peace and charity were instinctive virtues accident could not increase them
[NeMo I 2026-01-12 17:41:33 nemo_logging:393] 
    
[NeMo I 2026-01-12 17:41:33 nemo_loggi

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
from nemo.collections.asr.models import EncDecCTCModelBPE

model = EncDecCTCModelBPE.restore_from(save_model)
model.eval()



[NeMo I 2026-01-13 05:06:37 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2026-01-13 05:06:38 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: training.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 2
    
[NeMo W 2026-01-13 05:06:38 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: val.json
    sample_rate: 16000
    batch_size: 8
    shuffle: false
    num_workers: 2
    
[NeMo W 2026-01-13 05:06:38 nemo_logging:405] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    Test config : 
    manifest_filepath:

[NeMo I 2026-01-13 05:06:38 nemo_logging:393] PADDING: 0
[NeMo I 2026-01-13 05:06:38 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /content/drive/MyDrive/ASR/asr_conformer_finetuned.nemo.


EncDecCTCModelBPE(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConformerEncoder(
    (pre_encode): ConvSubsampling(
      (out): Linear(in_features=3520, out_features=176, bias=True)
      (conv): MaskedConvSequential(
        (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
    )
    (pos_enc): RelPositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-15): 16 x ConformerLayer(
        (norm_feed_forward1): LayerNorm((176,), eps=1e-05, elementwise_affine=True)
        (feed_forward1): ConformerFeedForward(
          (linear1): Linear(in_features=176, out_features=704, bias=True)
          (activation): Swish()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_feature

In [ ]:
pred=model.transcribe([test_audio])
print(pred[0])

[NeMo W 2026-01-13 05:06:55 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2026-01-13 05:06:55 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:03,  3.58s/it]

Hypothesis(score=tensor(-1.6549), y_sequence=tensor([1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,   32,
        1024,    9,    9,   12, 1024,  537, 1024, 1024, 1024, 1024, 1024, 1024,
          52, 1024, 1024, 1024, 1024,  564, 1024, 1024, 1024, 1024,    5, 1024,
        1024,  651, 1024, 1024, 1024, 1024, 1024, 1024,  160, 1024,   23, 1024,
        1024, 1024, 1024, 1024, 1024,    5, 1024, 1024, 1024,   23, 1024, 1024,
        1024,  534, 1024, 1024,  169, 1024, 1024, 1024, 1024, 1024, 1024,   10,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,   32,
        1024,    9,    9,   42, 1024,  189, 1024, 1024, 1024,   92, 1024, 1024,
           3, 1024, 1024,  324, 1024, 1024,  201, 1024, 1024, 1024,  233, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024]), text="we'd better not try to move himm tom decided we'll get an ambulance", dec_out=None, dec_state=None, timestamp=[], a